In [200]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from numpy import asarray
from datetime import datetime



print("Bibliothèques : OK")

df = pd.read_csv("../input/data-vac/donnees-vaccination-par-tranche-dage-type-de-vaccin-et-departement.csv",sep = ';')
df = df.dropna(axis=0)

df.drop(["libelle_region","libelle_departement","region_residence","libelle_classe_age","date"], axis = 1,inplace=True)


#CONVERTI STRING EN DATE
df["semaine_injection"] = pd.to_datetime(df["semaine_injection"]+'0', format='%Y-%W%w')
df["date_reference"] = pd.to_datetime(df["date_reference"], format='%Y-%m-%d')

df.set_index("semaine_injection",inplace=True)

filtre1 = df["departement_residence"] == "58"

df_nievre = df[filtre1]
df_nievre


In [201]:
df.duplicated().sum()

In [202]:
df_nievre.info()

In [203]:
df_nievre

In [204]:
print(df_nievre.shape)

df_nievre["effectif_termine"].describe()

In [205]:
df_nievre["classe_age"].value_counts()

# Taux de vaccination par tranche d'âge dans la Nièvre

In [206]:
df_nievre.groupby("classe_age")["taux_cumu_termine"].max()

MOYENNES PAR SEMAINE


Les plus de 55 ans représentent 45% de la population du département

# Taux de vaccination par tranche d'âge ET par vaccin dans la Nièvre

In [207]:
df_nievre.groupby(["classe_age","type_vaccin"]).max()

# Age et vaccin pour leurs donnéees maximums

In [208]:

df1 = df_nievre[df_nievre["classe_age"] == "18-24"]

df1.head()

#Max car le maximum de tous les vaccins sera la donnée "TOUT VACCIN" car somme des autres
df1.groupby("semaine_injection")["taux_cumu_termine"].max().plot(figsize=[20,10],label="18-24")



plt.title("Evolution du Taux de vaccination Totale de la population par semaine en 2021")
plt.ylabel("Taux de vaccination population")
plt.legend()

# Evolution du taux de vaccinnation complète des 18-24 ans dans la Nièvre par semaine de l'année 2021

In [209]:
df1.groupby("semaine_injection")["taux_cumu_termine"].max().plot.bar(figsize=[20,10])

plt.title("Evolution du Taux de vaccination Totale de la population par semaine en 2021")
plt.ylabel("Taux de vaccination population")
plt.legend()

# Analyse Générale 

In [210]:
df.groupby(["classe_age","type_vaccin"]).max()

# Analyse couverture vaccinnale complete chaque dpt (TOUTE POPULATION DONT LES < 12ANS)


In [211]:
dept = df["departement_residence"].unique().tolist()



tab_val = []
for i in range(len(dept)):
    #Cherche pour tous les ages et tous les vaccins à la dernière date possible
    df_tmp = df[(df["classe_age"] == "TOUT_AGE") & (df["departement_residence"] == dept[i]) & (df["type_vaccin"] == "Tout vaccin") & (df.index == pd.to_datetime(df.index.max(), format='%Y-%m-%d'))]
    tab_val.append(df_tmp.taux_cumu_termine.max())
    
df_gen_vacc = pd.DataFrame({'couv_vacc': tab_val},
                      index = dept)


# Departement plus faible taux de vaccination complète

In [212]:
df_gen_vacc[df_gen_vacc["couv_vacc"] == df_gen_vacc.couv_vacc.min()]

Le departement 973 (**La Guyane**) a la plus petite couverture vaccinale complète pour la population totale

# Departement plus fort taux de vaccination complète

In [213]:
df_gen_vacc[df_gen_vacc["couv_vacc"] == df_gen_vacc.couv_vacc.max()]

Le departement 85 (**La Vendée**) a la plus petite couverture complète vaccinale pour la population totale

In [214]:
print("Valeur de la couverture vaccinale de tous les départements annoncé par le Gouvernement : ",df_gen_vacc[df_gen_vacc.index == "Tout département"].values[0])
print("Valeur de la median : ",df_gen_vacc.couv_vacc.median())
print("Valeur de la moyenne : ",df_gen_vacc.couv_vacc.mean())

**/!\** Différence de **6.63%**

# Impact des annonces du président de la république sur la vaccination

In [215]:
# Conditions pour avoir DF Général
df_total = df[(df["classe_age"] == "TOUT_AGE") & (df["departement_residence"] == "Tout département") & (df["type_vaccin"] == "Tout vaccin") & (df.population_insee == 67287193.0)]

# 2 valeurs pour la première semaine donc suppression de la première
df_total = df_total[1:]

df_total



# Visualisation


In [216]:
df_total.groupby("semaine_injection")["taux_1_inj"].mean().plot(figsize=[20,10])
#df_total.groupby("semaine_injection")["taux_cumu_termine"].mean().plot(figsize=[20,10],color="green")
plt.axvline(x=pd.to_datetime("2021-07-12", format='%Y-%m-%d') ,color='red',linestyle='--',label="Allocution Présidentielle (Pass Sanitaire)")
plt.axvline(x=pd.to_datetime("2021-05-12", format='%Y-%m-%d') ,color='green',linestyle='--',label="Ouverture de la vaccination à tous (si stock disponible)")


plt.title("Evolution du taux de vaccination 1ère dose par semaine en France en 2021")
plt.ylabel("Taux de vaccination de la population")




plt.legend()

*  1er pic lors de l'ouverture de la vaccination à tous dans la limite des stocks restants
* 2ème pic lors de l'instauration du pass vaccinal (moins important que le premier, moins intense et moins long)


Descente logique début septembre car grande partie de la population vaccinée.

# Analyse de la vaccination totale par Age

In [217]:
# Conditions pour avoir DF Général
df_total_age = df[(df["departement_residence"] == "Tout département") & (df["type_vaccin"] == "Tout vaccin")]

# 2 valeurs pour la première semaine donc suppression de la première
#df_total_age = df_total[1:]


df_total_age.head()

In [218]:
cl_age = df["classe_age"].unique().tolist()

for i in range(len(l_age)):
    df_temp = df_total_age[df_total_age["classe_age"] == cl_age[i]]
    df_temp.groupby("semaine_injection")["taux_cumu_termine"].max().plot(figsize=(20,10), label=cl_age[i])


plt.axvline(x=pd.to_datetime("2021-07-12", format='%Y-%m-%d') ,color='red',linestyle='--',label="Allocution Présidentielle (Pass Sanitaire)")
plt.axvline(x=pd.to_datetime("2021-05-12", format='%Y-%m-%d') ,color='green',linestyle='--',label="Ouverture de la vaccination à tous (si stock disponible)")
plt.grid()
plt.legend()

**Lors de l'ouverture de la vaccinnation à tous, les +55 ans représentaient plus de la moitié des vaccinné(e)s en France**

**Lors de l'annonce de l'instauration du pass sanitaire, environ 45% des Francais(e) étaient vaccinné(e)s**


# Correlation Generale

In [219]:
plt.figure(figsize=(10, 10))
sns.heatmap(df.corr(), annot=True)

**La matrice de corrélation est coupée en 3 :**
* Un tier concernant la population brut (effectif) entre elle.
* Un tier concernant les taux entre eux.
* Un tier concernant les valeurs bruts et les taux.

**Les corrélations sont logiques :**
* Les données relatives à la popualtion (effectifs bruts) sont modérément corrélé positivement à la population : Plus il y a de personnes, plus il y a de personnes vaccinées.
* Très fortes corrélations positive entre les effectifs et taux cumulés entre la 1ère et la 2ème dosde : Les personne se faisant vaccinées 1 fois, le font une deuxieme fois.
* Aucune corrélation négative ou plus significative.